In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.preprocessing import LabelEncoder
from kmodes.kprototypes import KPrototypes
from matplotlib import style
from sklearn.preprocessing import MinMaxScaler

Import data

In [108]:
data = pd.read_csv('cleaned.csv')

C:\Users\G\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (11,18,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Copy the data in dataframe df

In [109]:
df=data.copy()
#df.head()

Keep data only for the year 2008 and 2009 (study period).

In [110]:
start=datetime.datetime(2008, 1, 1)
end=datetime.datetime(2009, 12, 31)

In [111]:
df['APPL_DATE']= pd.to_datetime(df['APPL_DATE'],dayfirst=True)
indices=[]
for i in range(len(df)):
    if i%100000==0:
        print(i)
    if df.iloc[i]['APPL_DATE']<start or df.iloc[i]['APPL_DATE']>end:
        indices.append(i)
df.drop(df.index[indices],inplace=True)

0
100000
200000
300000
400000
500000


Remove columns which we don't want our model to take

In [112]:
df.drop(['OD_AMT','ACTIVITY_NAME','CLIENT_SRLNO','UNIT_NAME','APPL_DATE','PRODUCT_NAME','PIN_CODE','REGIS_DATE','ASSET_BASE','VILLAGE_NAME','REPAY_TYPE','Unnamed: 0','FAMILY_MEN','FAMILY_WOM','NO_OF_INSTALL','CITY','ACTIVE_YN','TOTAL_INSTALLMENT_PAID'],axis=1,inplace=True)
#len(df)
#df.drop(['TOTAL_INSTALLMENT_PAID'],axis=1,inplace=True)
#df.drop(['PIN_CODE','REGIS_DATE','ASSET_BASE'],axis=1,inplace=True)
#df.isnull().sum()*100/df.shape[0]

Remove NaN's and convert values in the data to str or int

In [113]:
df.dropna(inplace=True)
df = df.astype({"VILLAGE_CODE": str,'FAMILY_MEM':int,'GUARANTOR':str,'CO_OBLIGANT':str,'CO_LATERAL':str,'CATEGORY':str,'GENDER':str})
#len(df)
#df.head()

In [114]:
#for i in range(len(df)):
#    if i%100000==0:
 #       print(i)
  #  if df.iloc[i]['ODDAYS']!=0:
  #      df.iloc[i]['ODDAYS']=1
#df.loc[df['ODDAYS'] > 0, 'First Season'] = 1
df['Default'] = (df['ODDAYS'] > 0).astype(int)

In [115]:
df.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,ODDAYS,Default
1865,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0.0,0
1866,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,156.0,1
1867,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,156.0,1
1868,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,156.0,1
1869,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,156.0,1


In [116]:
df.drop('ODDAYS',inplace=True,axis=1)

Encode the categorical variables. 
Firct convert the dataframe in a numpy array and then choose the columns to encode.

In [117]:
labelencoder1=LabelEncoder()

In [118]:
X=df.iloc[:,1:19].values
#X.shape

In [119]:
df.reset_index(inplace=True)

In [120]:
df.drop('index',axis=1,inplace=True)
#print(df.head())

In [121]:
df.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default
0,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0
1,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,1
2,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,1
3,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,1
4,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,1


In [122]:
for i in range (0,17):
    if i!=3 or i!=6 or i!=8 or i!=9 or i!=15:
        X[:,i]=labelencoder1.fit_transform(X[:,i])
        #print (i)

In [123]:
df.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default
0,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0
1,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,1
2,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,1
3,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,1
4,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,1


In [124]:
df2=df.copy()

Scaling of numerical variables.

In [125]:
scaler = MinMaxScaler()
df[['APPL_AMT', 'FAMILY_MEM','REGULAR','FESTIVE','AGE']] = scaler.fit_transform(df[['APPL_AMT', 'FAMILY_MEM','REGULAR','FESTIVE','AGE']])

C:\Users\G\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


ID is the identifier for each entry in out data

In [126]:
ID = df['ID_NO'].to_list()
ID= np.asarray(ID)
#len(ID)

In [127]:
#style.use("ggplot")
#colors = ['b', 'orange', 'g', 'r', 'c', 'm', 'y', 'k', 'Brown', 'ForestGreen']

In [128]:
print(X.shape)

(207274, 18)


Fitting the model k-prototype.
n_init = number of runs, 
categorical = column numbers of categorical variables, 
X = numpy array formed from the original data leaving the identifier column ID out, 
n_clusters = number of clusters

In [129]:
n_clust=4
kproto = KPrototypes(n_clusters=n_clust, init='Cao', verbose=2,n_init=1)
clusters = kproto.fit_predict(X, categorical=[0,1,3,4,6,9,10,11,12,13,15,16])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 33410, ncost: 207855766.1755184
Run: 1, iteration: 2/100, moves: 24888, ncost: 196311212.5387936
Run: 1, iteration: 3/100, moves: 11333, ncost: 193809104.25314668
Run: 1, iteration: 4/100, moves: 8503, ncost: 192156872.02773413
Run: 1, iteration: 5/100, moves: 8922, ncost: 190554292.84243065
Run: 1, iteration: 6/100, moves: 11027, ncost: 187561044.56955534
Run: 1, iteration: 7/100, moves: 8771, ncost: 186170868.37380004
Run: 1, iteration: 8/100, moves: 14572, ncost: 174525699.65755728
Run: 1, iteration: 9/100, moves: 30542, ncost: 159639218.553865
Run: 1, iteration: 10/100, moves: 6805, ncost: 158911708.83067748
Run: 1, iteration: 11/100, moves: 3051, ncost: 158677298.0155168
Run: 1, iteration: 12/100, moves: 4711, ncost: 158107557.58206758
Run: 1, iteration: 13/100, moves: 1910, ncost: 157958014.16740182
Run: 1, iteration: 14/100, moves: 319, ncost: 157952627.56567317
Run: 

Location of the centroids

In [42]:
#print(kproto.cluster_centroids_)

Save the obtained clusters in a dictionary where keys are ID's and values are the cluster numbers. Also, append the cluster numbers to original dataframe.

In [134]:
clust_dict={}
clust=[]
for s, c in zip(ID, clusters):
    clust_dict[s]=c
    clust.append(c)
df2['cluster']=clust

In [135]:
df2.to_csv('cleaned_with_clusters_vs_ID.csv')

In [136]:
df2.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default,cluster
0,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0,2
1,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,1,0
2,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,1,0
3,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,1,0
4,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,1,3


In [137]:
for i in range(0,n_clust):
    print ("Cluster",i,":", df2.groupby('cluster').count().iloc[i]['ID_NO'])
#df.groupby('cluster').count()

Cluster 0 : 66829
Cluster 1 : 25297
Cluster 2 : 81408
Cluster 3 : 33740


In [138]:
summary = pd.read_csv('cleaned_with_clusters_vs_ID.csv')

In [139]:
summary.groupby('cluster').count()

,Unnamed: 0,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default
cluster,,,,,,,,,,,,,,,,,,,,
0,66829,66829,66829,66829,66829,66829,66829,66829,66829,66829,66829,66829,66829,66829,66829,66829,66829,66829,66829,66829
1,25297,25297,25297,25297,25297,25297,25297,25297,25297,25297,25297,25297,25297,25297,25297,25297,25297,25297,25297,25297
2,81408,81408,81408,81408,81408,81408,81408,81408,81408,81408,81408,81408,81408,81408,81408,81408,81408,81408,81408,81408
3,33740,33740,33740,33740,33740,33740,33740,33740,33740,33740,33740,33740,33740,33740,33740,33740,33740,33740,33740,33740


In [140]:
summary.drop('Unnamed: 0',inplace=True,axis=1)

Based on category

In [141]:
print('Clusterwise category for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].index.tolist())
        print(count)
        
df1 = summary.groupby('CATEGORY').count()
df1.reset_index(inplace=True)
#df1.head()
df1 = df1[['CATEGORY','ID_NO']].copy()

print('Category-wise summary for all clusters:\n')
for cat in summary['CATEGORY'].unique().tolist():
    print('\nFor category',cat,'the total number of applications are-',df1[df1['CATEGORY']==cat]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].tolist())
        cats = summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].index.tolist()
        df_dummy = pd.DataFrame({'CAT':cats,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['CAT']==cat]['count']*100/float(df1[df1['CATEGORY']==cat]['ID_NO'].to_string(index=None)))
            

Clusterwise category for all clusters

Cluster 0 :-
['O.B.C', 'S.C', 'MINORITY', 'S.T', 'O.C']
[51.6, 26.05, 12.52, 5.35, 4.47]
Cluster 1 :-
['O.B.C', 'MINORITY', 'S.C', 'O.C', 'S.T']
[53.65, 20.63, 18.61, 5.24, 1.87]
Cluster 2 :-
['O.B.C', 'S.C', 'MINORITY', 'S.T', 'O.C']
[48.94, 25.49, 16.31, 4.67, 4.59]
Cluster 3 :-
['O.B.C', 'S.C', 'MINORITY', 'S.T', 'O.C']
[52.29, 26.37, 12.8, 5.07, 3.46]
Category-wise summary for all clusters:


For category O.B.C the total number of applications are-  105541
Cluster 0 has below %
0    32.673558
Name: count, dtype: float64
Cluster 1 has below %
0    12.859457
Name: count, dtype: float64
Cluster 2 has below %
0    37.749311
Name: count, dtype: float64
Cluster 3 has below %
0    16.717674
Name: count, dtype: float64

For category S.C the total number of applications are-  51767
Cluster 0 has below %
1    33.633396
Name: count, dtype: float64
Cluster 1 has below %
2    9.094597
Name: count, dtype: float64
Cluster 2 has below %
1    40.085383
Name: c

Based on default behaviour

In [142]:
print('Clusterwise defualt behaviour\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['Default'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['Default'].value_counts()[:2].index.tolist())
        print(count)
        
df1 = summary.groupby('Default').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['Default','ID_NO']].copy()

print('Default behaviour wise summary for all clusters:\n')
for default in summary['Default'].unique().tolist():
    print('\nFor default',default,'the total number of applications are-',df1[df1['Default']==default]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['Default'].value_counts()[:2].tolist())
        defs = summary[summary['cluster']==i]['Default'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Def':defs,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Def']==default]['count']*100/float(df1[df1['Default']==default]['ID_NO'].to_string(index=None)))
            

Clusterwise defualt behaviour

Cluster 0 :-
[0, 1]
[63.46, 36.54]
Cluster 1 :-
[0, 1]
[68.17, 31.83]
Cluster 2 :-
[0, 1]
[64.61, 35.39]
Cluster 3 :-
[0, 1]
[63.88, 36.12]
Default behaviour wise summary for all clusters:


For default 0 the total number of applications are-  133807
Cluster 0 has below %
0    31.692662
Name: count, dtype: float64
Cluster 1 has below %
0    12.888713
Name: count, dtype: float64
Cluster 2 has below %
0    39.31035
Name: count, dtype: float64
Cluster 3 has below %
0    16.108275
Name: count, dtype: float64

For default 1 the total number of applications are-  73467
Cluster 0 has below %
1    33.242136
Name: count, dtype: float64
Cluster 1 has below %
1    10.958662
Name: count, dtype: float64
Cluster 2 has below %
1    39.212163
Name: count, dtype: float64
Cluster 3 has below %
1    16.587039
Name: count, dtype: float64


Based on Secured/Unsecured status

In [143]:
print('Clusterwise secured/unsecured for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:5].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:5].index.tolist())
        print(count)
        
df1 = summary.groupby('UNSECURE_SECURE').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['UNSECURE_SECURE','ID_NO']].copy()


print('Secured/unsecured wise summary for all clusters:\n')
for status in summary['UNSECURE_SECURE'].unique().tolist():
    print('\nFor status',status,'the total number of applications are-',df1[df1['UNSECURE_SECURE']==status]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:2].tolist())
        statuses = summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'STATUS':statuses,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['STATUS']==status]['count']*100/float(df1[df1['UNSECURE_SECURE']==status]['ID_NO'].to_string(index=None)))
            

Clusterwise secured/unsecured for all clusters

Cluster 0 :-
['U', 'S']
[100.0, 0.0]
Cluster 1 :-
['U', 'S']
[99.99, 0.01]
Cluster 2 :-
['U', 'S']
[100.0, 0.0]
Cluster 3 :-
['U', 'S']
[100.0, 0.0]
Secured/unsecured wise summary for all clusters:


For status U the total number of applications are-  207268
Cluster 0 has below %
0    32.242314
Name: count, dtype: float64
Cluster 1 has below %
0    12.203524
Name: count, dtype: float64
Cluster 2 has below %
0    39.276203
Name: count, dtype: float64
Cluster 3 has below %
0    16.277959
Name: count, dtype: float64

For status S the total number of applications are-  6
Cluster 0 has below %
1    16.666667
Name: count, dtype: float64
Cluster 1 has below %
1    50.0
Name: count, dtype: float64
Cluster 2 has below %
1    16.666667
Name: count, dtype: float64
Cluster 3 has below %
1    16.666667
Name: count, dtype: float64


Based on house type

In [144]:
summary.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default,cluster
0,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0,2
1,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,1,0
2,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,1,0
3,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,1,0
4,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,1,3


In [145]:
#Based on house type
print('Clusterwise house type for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].index.tolist())
        print(count)        
        
df1 = summary.groupby('HOUSE_TYPE').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['HOUSE_TYPE','ID_NO']].copy()


print('House type wise summary for all clusters:\n')
for htype in summary['HOUSE_TYPE'].unique().tolist():
    print('\nFor house type',htype,'the total number of applications are-',df1[df1['HOUSE_TYPE']==htype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].tolist())
        types = summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].index.tolist()
        df_dummy = pd.DataFrame({'HOUSE':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['HOUSE']==htype]['count']*100/float(df1[df1['HOUSE_TYPE']==htype]['ID_NO'].to_string(index=None)))
            

Clusterwise house type for all clusters

Cluster 0 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets', 'Stones']
[49.58, 29.41, 14.31, 4.91, 1.78]
Cluster 1 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets', 'Stones']
[61.65, 23.75, 8.73, 3.95, 1.92]
Cluster 2 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets', 'Stones']
[48.98, 28.29, 14.96, 5.47, 2.3]
Cluster 3 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets', 'Stones']
[47.42, 30.23, 14.71, 4.51, 3.13]
House type wise summary for all clusters:


For house type Tiles the total number of applications are-  58893
Cluster 0 has below %
1    33.377481
Name: count, dtype: float64
Cluster 1 has below %
1    10.20325
Name: count, dtype: float64
Cluster 2 has below %
1    39.101421
Name: count, dtype: float64
Cluster 3 has below %
1    17.317848
Name: count, dtype: float64

For house type Tatched the total number of applications are-  104606
Cluster 0 has below %
0    31.676959
Name: count, dtype: float64
Cluster 1 has below %
0    14.908323
Name: count, dtype: float64
Clus

Based on Guarantor

In [146]:
print('Clusterwise guarantor for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].index.tolist())
        print(count)        
        
df1 = summary.groupby('GUARANTOR').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['GUARANTOR','ID_NO']].copy()


print('Gurantor based summary for all clusters:\n')
for gtype in summary['GUARANTOR'].unique().tolist():
    print('\nFor guarantor',gtype,'the total number of applications are-',df1[df1['GUARANTOR']==htype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Guarantor':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Guarantor']==gtype]['count']*100/float(df1[df1['GUARANTOR']==gtype]['ID_NO'].to_string(index=None)))
            

Clusterwise guarantor for all clusters

Cluster 0 :-
['N', 'G']
[100.0, 0.0]
Cluster 1 :-
['N']
[100.0]
Cluster 2 :-
['N']
[100.0]
Cluster 3 :-
['N']
[100.0]
Gurantor based summary for all clusters:


For guarantor N the total number of applications are- Series([], )
Cluster 0 has below %
0    32.240883
Name: count, dtype: float64
Cluster 1 has below %
0    12.204795
Name: count, dtype: float64
Cluster 2 has below %
0    39.276117
Name: count, dtype: float64
Cluster 3 has below %
0    16.278206
Name: count, dtype: float64

For guarantor G the total number of applications are- Series([], )
Cluster 0 has below %
1    100.0
Name: count, dtype: float64
Cluster 1 has below %
Series([], Name: count, dtype: float64)
Cluster 2 has below %
Series([], Name: count, dtype: float64)
Cluster 3 has below %
Series([], Name: count, dtype: float64)


In [147]:
#Based on Co obligant

print('Clusterwise co obligant for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('CO_OBLIGANT').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['CO_OBLIGANT','ID_NO']].copy()


print('Guarantor based summary for all clusters:\n')
for otype in summary['CO_OBLIGANT'].unique().tolist():
    print('\nFor co obligant',otype,'the total number of applications are-',df1[df1['CO_OBLIGANT']==otype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Obligant':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Obligant']==otype]['count']*100/float(df1[df1['CO_OBLIGANT']==otype]['ID_NO'].to_string(index=None)))
            

Clusterwise co obligant for all clusters

Cluster 0 :-
['N', 'O']
[99.03, 0.97]
Cluster 1 :-
['N', 'O']
[99.05, 0.95]
Cluster 2 :-
['N', 'O']
[98.88, 1.12]
Cluster 3 :-
['N', 'O']
[99.17, 0.83]
Guarantor based summary for all clusters:


For co obligant N the total number of applications are-  205193
Cluster 0 has below %
0    32.254024
Name: count, dtype: float64
Cluster 1 has below %
0    12.21143
Name: count, dtype: float64
Cluster 2 has below %
0    39.228434
Name: count, dtype: float64
Cluster 3 has below %
0    16.306112
Name: count, dtype: float64

For co obligant O the total number of applications are-  2081
Cluster 0 has below %
1    31.042768
Name: count, dtype: float64
Cluster 1 has below %
1    11.532917
Name: count, dtype: float64
Cluster 2 has below %
1    43.921192
Name: count, dtype: float64
Cluster 3 has below %
1    13.503123
Name: count, dtype: float64


In [148]:
#Based on Collateral

print('Clusterwise collateral for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('CO_LATERAL').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['CO_LATERAL','ID_NO']].copy()


print('\nCollateral based summary for all clusters:')
for otype in summary['CO_LATERAL'].unique().tolist():
    print('\nFor collateral',otype,'the total number of applications are-',df1[df1['CO_LATERAL']==otype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Collateral':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Collateral']==otype]['count']*100/float(df1[df1['CO_LATERAL']==otype]['ID_NO'].to_string(index=None)))
            

Clusterwise collateral for all clusters

Cluster 0 :-
['N', 'L']
[100.0, 0.0]
Cluster 1 :-
['N', 'L']
[99.99, 0.01]
Cluster 2 :-
['N', 'L']
[100.0, 0.0]
Cluster 3 :-
['N', 'L']
[100.0, 0.0]

Collateral based summary for all clusters:

For collateral N the total number of applications are-  207268
Cluster 0 has below %
0    32.242314
Name: count, dtype: float64
Cluster 1 has below %
0    12.203524
Name: count, dtype: float64
Cluster 2 has below %
0    39.276203
Name: count, dtype: float64
Cluster 3 has below %
0    16.277959
Name: count, dtype: float64

For collateral L the total number of applications are-  6
Cluster 0 has below %
1    16.666667
Name: count, dtype: float64
Cluster 1 has below %
1    50.0
Name: count, dtype: float64
Cluster 2 has below %
1    16.666667
Name: count, dtype: float64
Cluster 3 has below %
1    16.666667
Name: count, dtype: float64


In [149]:
#Based on Gender

print('Clusterwise gender for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['GENDER'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['GENDER'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('GENDER').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['GENDER','ID_NO']].copy()


print('\nGender based summary for all clusters:')
for otype in summary['GENDER'].unique().tolist():
    print('\nFor gender',otype,'the total number of applications are-',df1[df1['GENDER']==otype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['GENDER'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['GENDER'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'gender':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['gender']==otype]['count']*100/float(df1[df1['GENDER']==otype]['ID_NO'].to_string(index=None)))
            

Clusterwise gender for all clusters

Cluster 0 :-
['F', 'M']
[73.19, 26.81]
Cluster 1 :-
['F', 'M']
[82.74, 17.26]
Cluster 2 :-
['F', 'M']
[75.38, 24.62]
Cluster 3 :-
['F', 'M']
[74.53, 25.47]

Gender based summary for all clusters:

For gender F the total number of applications are-  156353
Cluster 0 has below %
0    31.281779
Name: count, dtype: float64
Cluster 1 has below %
0    13.387015
Name: count, dtype: float64
Cluster 2 has below %
0    39.247088
Name: count, dtype: float64
Cluster 3 has below %
0    16.084117
Name: count, dtype: float64

For gender M the total number of applications are-  50921
Cluster 0 has below %
1    35.189804
Name: count, dtype: float64
Cluster 1 has below %
1    8.574066
Name: count, dtype: float64
Cluster 2 has below %
1    39.362935
Name: count, dtype: float64
Cluster 3 has below %
1    16.873196
Name: count, dtype: float64


In [150]:
summary.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default,cluster
0,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0,2
1,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,1,0
2,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,1,0
3,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,1,0
4,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,1,3


In [151]:
#Based on Accomodation

print('Clusterwise accomodation for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('ACCOMADATION').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['ACCOMADATION','ID_NO']].copy()


print('\nAccomodation based summary for all clusters:')
for otype in summary['ACCOMADATION'].unique().tolist():
    print('\nFor accomodation',otype,'the total number of applications are-',df1[df1['ACCOMADATION']==otype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'acco':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['acco']==otype]['count']*100/float(df1[df1['ACCOMADATION']==otype]['ID_NO'].to_string(index=None)))
            

Clusterwise accomodation for all clusters

Cluster 0 :-
['O', 'R']
[70.2, 29.8]
Cluster 1 :-
['O', 'R']
[57.01, 42.99]
Cluster 2 :-
['O', 'R']
[66.3, 33.7]
Cluster 3 :-
['O', 'R']
[76.02, 23.98]

Accomodation based summary for all clusters:

For accomodation R the total number of applications are-  66318
Cluster 0 has below %
1    30.031062
Name: count, dtype: float64
Cluster 1 has below %
1    16.396755
Name: count, dtype: float64
Cluster 2 has below %
1    41.371875
Name: count, dtype: float64
Cluster 3 has below %
1    12.200308
Name: count, dtype: float64

For accomodation O the total number of applications are-  140956
Cluster 0 has below %
0    33.282017
Name: count, dtype: float64
Cluster 1 has below %
0    10.232271
Name: count, dtype: float64
Cluster 2 has below %
0    38.289253
Name: count, dtype: float64
Cluster 3 has below %
0    18.196458
Name: count, dtype: float64


SUMMARY STATS FOR CONTINUOUS NUMERICAL VARIABLES: APPL_AMT, FAMILY_MEM, REGULAR, FESTIVE, AGE

In [154]:
summary.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default,cluster
0,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0,2
1,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,1,0
2,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,1,0
3,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,1,0
4,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,1,3


In [165]:
for i in range(0,n_clust):
    sliced = summary[summary['cluster']==i]
    print('Stats for cluster',i,':-')
    print('1. Application amount-','Mean:',round(sliced['APPL_AMT'].mean(),ndigits=2),',Min:',round(sliced['APPL_AMT'].min(),ndigits=2),',Max:',round(sliced['APPL_AMT'].max(),ndigits=2),',Median:',round(sliced['APPL_AMT'].median(),ndigits=2))
    print('2. Family members-','Mean:',round(sliced['FAMILY_MEM'].mean(),ndigits=2),',Min:',round(sliced['FAMILY_MEM'].min(),ndigits=2),',Max:',round(sliced['FAMILY_MEM'].max(),ndigits=2),',Median:',round(sliced['FAMILY_MEM'].median(),ndigits=2))
    print('3. Regular-','Mean:',round(sliced['REGULAR'].mean(),ndigits=2),',Min:',round(sliced['REGULAR'].min(),ndigits=2),',Max:',round(sliced['REGULAR'].max(),ndigits=2),',Median:',round(sliced['REGULAR'].median(),ndigits=2))
    print('4. Festive-','Mean:',round(sliced['FESTIVE'].mean(),ndigits=2),',Min:',round(sliced['FESTIVE'].min(),ndigits=2),',Max:',round(sliced['FESTIVE'].max(),ndigits=2),',Median:',round(sliced['FESTIVE'].median(),ndigits=2))
    print('5. Age-','Mean:',round(sliced['AGE'].mean(),ndigits=2),',Min:',round(sliced['AGE'].min(),ndigits=2),',Max:',round(sliced['AGE'].max(),ndigits=2),',Median:',round(sliced['AGE'].median(),ndigits=2))
    print('\n')

Stats for cluster 0 :-
1. Application amount- Mean: 14230.53 ,Min: 5000.0 ,Max: 26042009.0 ,Median: 15000.0
2. Family members- Mean: 2.54 ,Min: 1 ,Max: 11 ,Median: 2.0
3. Regular- Mean: 1168.99 ,Min: 3.0 ,Max: 60070.0 ,Median: 1000.0
4. Festive- Mean: 1861.59 ,Min: 700.0 ,Max: 3500.0 ,Median: 2000.0
5. Age- Mean: 33.58 ,Min: 18.0 ,Max: 66.0 ,Median: 33.0


Stats for cluster 1 :-
1. Application amount- Mean: 13650.52 ,Min: 5000.0 ,Max: 15082009.0 ,Median: 12000.0
2. Family members- Mean: 2.9 ,Min: 1 ,Max: 13 ,Median: 2.0
3. Regular- Mean: 3910.32 ,Min: 50.0 ,Max: 800000.0 ,Median: 3500.0
4. Festive- Mean: 325605.78 ,Min: 3500.0 ,Max: 8000000000.0 ,Median: 8000.0
5. Age- Mean: 34.68 ,Min: 18.0 ,Max: 63.0 ,Median: 34.0


Stats for cluster 2 :-
1. Application amount- Mean: 13447.31 ,Min: 5000.0 ,Max: 902207.0 ,Median: 12000.0
2. Family members- Mean: 2.79 ,Min: 1 ,Max: 14 ,Median: 2.0
3. Regular- Mean: 1907.12 ,Min: 50.0 ,Max: 414000.0 ,Median: 2000.0
4. Festive- Mean: 3578.76 ,Min: 1500.0